In [1]:

import pickle
import pandas as pd
import numpy as np


import torch as tc
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


/home/shao.367/miniconda3/envs/pt-gpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

with open('data/data_res.pickle', 'rb') as pk:
    data = pickle.load(pk)

In [3]:
from collections import defaultdict
data_reorg = defaultdict(list)

with open('data/preprocessing.pickle', 'rb') as pk:
    data_preprocessing = pickle.load(pk)

user_class = data_preprocessing['user_class']

for user in data['x']:
    for i,game in enumerate(data['x'][user]):
        if i+1 <= len(data['x'][user]) - 1:
            data_reorg['x_t'].append( data['x'][user][i] )
            data_reorg['y_t'].append( data['y'][user][i] )
            data_reorg['u_t'].append( data['u'][user][i] )
            data_reorg['x_t1'].append( data['x'][user][i+1] )
            data_reorg['y_t1'].append( data['y'][user][i+1] )
            data_reorg['class'].append( user_class[user][0] )
            
df = pd.DataFrame( data_reorg )


In [4]:

TRAINING_SAMPLE_SIZE = int(0.75*len(df))
train_idx = np.random.randint(0, len(df)-1, TRAINING_SAMPLE_SIZE)
verify_idx = np.setdiff1d(np.arange(len(df)), train_idx)

df.head()

,x_t,y_t,u_t,x_t1,y_t1,class
0,0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...",20
1,0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...",20
2,0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...",20
3,0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.086603,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...",20
4,0.086603,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",0.100000,"[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...",20


In [5]:

class Net(tc.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = tc.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.predict = tc.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x

train_data = df.iloc[train_idx]
verify_data = df.iloc[verify_idx]
x_dim = 1
y_dim = len(train_data['y_t'].iloc[0])
u_dim = len(train_data['u_t'].iloc[0])

h_input_layer_dim = x_dim+u_dim
nn_hidden_width = 100
h_output_layer_dim = y_dim


In [6]:
x_set, u_set = np.asarray(train_data['x_t'].to_list()), np.asarray(train_data['u_t'].to_list())
y_set = np.asarray(train_data['y_t'].to_list())
x_set = np.reshape(x_set, (len(x_set), 1))
h_input_data = tc.tensor(np.hstack([x_set, u_set]), dtype=tc.float)
h_output_data = tc.tensor(y_set, dtype=tc.float)

In [7]:

net = nn.Sequential(nn.Linear(h_input_layer_dim, nn_hidden_width), 
                    nn.ReLU(), nn.Linear(nn_hidden_width, nn_hidden_width),
                    nn.ReLU(), nn.Linear(nn_hidden_width, nn_hidden_width),
                    nn.ReLU(), nn.Linear(nn_hidden_width, h_output_layer_dim))

optimizer = tc.optim.Adam(net.parameters(), lr=0.005)
loss_func = tc.nn.MSELoss()  # this is for regression mean squared loss


for _ in range(10):
    optimizer.zero_grad()                 # clear gradients for next train
    v_pred = net(h_input_data)            # input x and predict based on x  
    loss = loss_func(v_pred, h_output_data)   # must be (1. nn output, 2. target)
    
    loss.backward()                            # backpropagation, compute gradients
    optimizer.step()                      # apply gradients
    print(f'eps: {_} -- {loss}')

eps: 0 -- 0.007037163712084293
eps: 1 -- 0.003481911960989237
eps: 2 -- 0.0017806917894631624
eps: 3 -- 0.0012653067242354155
eps: 4 -- 0.000676391355227679
eps: 5 -- 0.0005318347830325365
eps: 6 -- 0.0005579803255386651
eps: 7 -- 0.0005771802389062941
eps: 8 -- 0.0005807061679661274
eps: 9 -- 0.0005769491544924676


In [8]:
x_set, u_set = np.asarray(verify_data['x_t'].to_list()), np.asarray(verify_data['u_t'].to_list())
y_set = np.asarray(verify_data['y_t'].to_list())
x_set = np.reshape(x_set, (len(x_set), 1))
h_input_verify = tc.tensor(np.hstack([x_set, u_set]), dtype=tc.float)
h_output_verify = tc.tensor(y_set, dtype=tc.float)

In [9]:
v_pred = net(h_input_verify)
loss = loss_func(v_pred, h_output_verify)
print(loss)

tensor(0.0006, grad_fn=<MseLossBackward>)


In [10]:
# f1
user_data = defaultdict(list)
cnt = 0
for i,row in df.iterrows():
    x = row.loc['x_t']
    y = row.loc['y_t']
    u = row.loc['u_t']
    x_next = row.loc['x_t1']

    
    if np.linalg.norm(y) != 0 and np.linalg.norm(u) != 0 and np.dot(y, u) != 0:
        ep = (x_next-x) / (np.dot(y, u)/(np.linalg.norm(y)*np.linalg.norm(u)))
    else:
        ep = 0

    user_data[row.loc['class']].append(ep)

#     if not (np.linalg.norm(y) != 0 and np.linalg.norm(u) != 0):
#         print(i, y, u)
#         cnt += 1
# print(cnt)

In [17]:
epsion_user = np.zeros(len(list(user_data.keys())))
for i,user in enumerate(user_data.keys()):
    epsion_user[i] = np.array(user_data[user]).mean()

epsion_user = np.clip(epsion_user, a_min=0, a_max=np.inf)
print(epsion_user)

with open(f'data/epsilon.pickle', 'wb') as pk:
    pickle.dump(epsion_user, pk, protocol=pickle.HIGHEST_PROTOCOL) 

[1.64682487 1.6397198  0.90589726 0.52088791 1.36371071 0.97572749
 3.04661257 0.89093565 3.7176052  1.65594446 1.55603544 2.28185716
 4.73746149 0.81728338 2.36801248 1.27636493 2.29618325 0.94048599
 0.26732503 4.30048655 0.12414949 1.66147287 0.21128627 2.36259984
 0.09071123 3.69853523 3.66003637 0.87443074 1.62936286 1.55688002
 1.09509216 0.66426975 1.11240794 0.48146877 0.60147965 0.94582615
 0.29152373 3.82517031 0.42158417 0.23206823 0.05671402 0.23372655
 0.         0.54954319 1.40483824 0.14008946 0.11828592 1.33986892
 0.0137335  0.02802405]


In [12]:
# fit f2
f2_input_data = defaultdict(list)
f2_output_data = defaultdict(list)
cnt = 0
for i,row in df.iterrows():
    x = row.loc['x_t']
    y = row.loc['y_t']
    u = row.loc['u_t']
    y_next = row.loc['y_t1']

    input_data = np.hstack([x, y, u])

    f2_input_data[row.loc['class']].append(input_data)
    f2_output_data[row.loc['class']].append(y_next)



In [13]:
f2_input_layer_dim = len(f2_input_data[0][0])
f2_output_layer_dim = len(f2_output_data[0][0])
f2_hidden_width = f2_input_layer_dim
for i in f2_input_data:
    f2_input_data[i] = tc.tensor(np.array(f2_input_data[i]), dtype=tc.float)
    f2_output_data[i] = tc.tensor(np.array(f2_output_data[i]), dtype=tc.float)

In [14]:
from tqdm import tqdm
f2_nn_dict = {}
W_SCALE = 5

for i in tqdm(f2_input_data):

    net = nn.Sequential(nn.Linear(f2_input_layer_dim, W_SCALE*f2_hidden_width), 
                    nn.ReLU(), nn.Linear(W_SCALE*f2_hidden_width, W_SCALE*f2_hidden_width),
                    nn.ReLU(), nn.Linear(W_SCALE*f2_hidden_width, f2_output_layer_dim),
                    nn.Softmax())

    optimizer = tc.optim.Adam(net.parameters(), lr=0.002)
    loss_func = tc.nn.MSELoss()  # this is for regression mean squared loss


    for _ in tqdm(range(1000)):
        optimizer.zero_grad()                 # clear gradients for next train
        v_pred = net(f2_input_data[0])            # input x and predict based on x  
        loss = loss_func(v_pred, f2_output_data[0])   # must be (1. nn output, 2. target)
        
        loss.backward()                            # backpropagation, compute gradients
        optimizer.step()                      # apply gradients
        # if _%100 == 0:
        #     print(f'eps: {_} -- {loss}')

    f2_nn_dict[i] = net.state_dict()

tc.save(f2_nn_dict, 'data/f2_nn')


  0%|          | 0/50 [00:00<?, ?it/s]/home/shao.367/miniconda3/envs/pt-gpu/lib/python3.9/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 50/50 [10:29<00:00, 12.60s/it]


In [15]:
f2_nn_dict = tc.load('data/f2_nn')
# verify
cnt = 0
# predict
for i,row in df.iterrows():
    x = row.loc['x_t']
    y = row.loc['y_t']
    u = row.loc['u_t']
    y_next = row.loc['y_t1']
    user_c = row.loc['class']
    
    cnt += 1
    if cnt == 1002:
        break

net_predit = net
net_predit.load_state_dict(f2_nn_dict[user_c])
input_data = tc.tensor(np.hstack([x, y, u]), dtype=tc.float)
y_next = tc.tensor(y_next, dtype=tc.float)
out_y = net_predit(input_data)
print(loss_func(out_y, y_next) )


tensor(0.0003, grad_fn=<MseLossBackward>)
